# LME Copper Market Tightness Analysis

This notebook ingests daily LME copper cash–3M spread and visible warehouse stocks, computes a simple tightness index using z‑scores, and visualises the relationship between spreads and stocks. The analysis also classifies regimes (Tight, Neutral, Loose) and produces an index for the latest observation.

In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt

# Read data
df = pd.read_csv('data/lme_copper.csv', parse_dates=['date']).sort_values('date').reset_index(drop=True)

# Compute z‑scores
spread_mean, spread_std = df['cash_3m_spread_usd_t'].mean(), df['cash_3m_spread_usd_t'].std(ddof=0)
stock_mean, stock_std = df['stocks_tonnes'].mean(), df['stocks_tonnes'].std(ddof=0)

# Backwardation positive, contango negative
df['z_cash3m'] = (df['cash_3m_spread_usd_t'] - spread_mean) / (spread_std if spread_std!=0 else 1)
# Invert stocks so low stocks = high tightness
df['z_stocks'] = -(df['stocks_tonnes'] - stock_mean) / (stock_std if stock_std!=0 else 1)

# Tightness index and regime classification
df['tightness_index'] = df['z_cash3m'] + df['z_stocks']
df['regime'] = np.where(df['tightness_index']>1,'Tight',np.where(df['tightness_index']<-1,'Loose','Neutral'))


In [ ]:
# Optional: rolling 30‑day correlation between changes in spreads and stocks
if len(df) >= 31:
    df['spread_change'] = df['cash_3m_spread_usd_t'].diff()
    df['stock_change'] = df['stocks_tonnes'].diff()
    df['rolling_corr'] = df['spread_change'].rolling(window=30).corr(df['stock_change'])
else:
    df['rolling_corr'] = np.nan

# Plot time series and scatter
fig, axs = plt.subplots(2, 1, figsize=(10, 8), gridspec_kw={'height_ratios':[2,1]})

# Top: cash‑3M spread vs stocks (two axes)
ax = axs[0]
ax.plot(df['date'], df['cash_3m_spread_usd_t'], color='tab:blue', label='Cash‑3M spread (USD/t)')
ax.axhline(0, color='gray', ls='--', lw=0.8)
ax.set_ylabel('Cash‑3M Spread (USD/t)', color='tab:blue')
ax.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax.twinx()
ax2.plot(df['date'], df['stocks_tonnes'], color='tab:red', label='Visible stocks (t)')
ax2.set_ylabel('Visible Stocks (t)', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')
ax.set_title('LME Copper Cash‑3M Spread vs Stocks')

# Shade tight/loose regimes
for regime, color in [('Tight',(0,0.6,0,0.15)),('Loose',(1,0,0,0.15))]:
    mask = df['regime'] == regime
    i=0
    while i < len(mask):
        if mask.iloc[i]:
            start=i
            while i+1 < len(mask) and mask.iloc[i+1]: i+=1
            end=i
            ax.axvspan(df.loc[start,'date'], df.loc[end,'date'], color=color)
        i+=1

# Bottom: scatter
ax3 = axs[1]
scatter = ax3.scatter(df['stocks_tonnes'], df['cash_3m_spread_usd_t'], c=df['tightness_index'], cmap='RdYlGn', edgecolor='black')
coeffs = np.polyfit(df['stocks_tonnes'], df['cash_3m_spread_usd_t'], 1)
ax3.plot(df['stocks_tonnes'], np.polyval(coeffs, df['stocks_tonnes']), color='black', ls='--', lw=1)
last = df.iloc[-1]
ax3.scatter(last['stocks_tonnes'], last['cash_3m_spread_usd_t'], color='black', s=60, label='Today')
ax3.set_xlabel('Visible Stocks (t)')
ax3.set_ylabel('Cash‑3M Spread (USD/t)')
ax3.set_title('Scatter: Stocks vs Cash‑3M Spread')
ax3.legend(loc='best')

plt.tight_layout()
plt.show()


In [ ]:
# Save outputs
# Figure saved above can be exported here as an example
fig.savefig('lme_copper_tightness.png', dpi=150)
# Save today’s row
last[['date','cash_3m_spread_usd_t','stocks_tonnes','z_cash3m','z_stocks','tightness_index','regime']].to_frame().T.to_csv('lme_copper_today.csv', index=False)
